In [22]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [23]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 2, doc_content_chars_max = 500)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)
arxiv.name

'arxiv'

In [24]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 2, doc_content_chars_max = 500)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ReAct Agent"

In [27]:
def add(a:int,b:int) -> int:
    '''
    Add the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a + b

def multiply(a:int,b:int) -> int:
    '''
    Multiply the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a * b

def div(a:int,b:int) -> float:
    '''
    Divide the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a / b

In [28]:
tools = [arxiv, wiki, add, multiply, div]

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o", temperature = 0)

llm_with_tools = llm.bind_tools(tools)

In [30]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [31]:
def tool_calling_llm(state:State):
    return{"messages":[llm_with_tools.invoke(state["messages"])]}

In [32]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

In [33]:
from langchain_core.messages import HumanMessage

messages = graph.invoke({"messages":HumanMessage(content = "What is attention?, then add 5 plus 5 and then multiply by 3 and then 10")})

for m in messages["messages"]:
    m.pretty_print()


================================ Human Message =================================

What is attention?, then add 5 plus 5 and then multiply by 3 and then 10
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_fxnpa3Xr4AWyyuomLSACXEfD)
 Call ID: call_fxnpa3Xr4AWyyuomLSACXEfD
  Args:
    query: Attention (machine learning)
================================= Tool Message =================================
Name: wikipedia

Page: Attention (machine learning)
Summary: In machine learning, attention is a method that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is represented by "soft" weights assigned to each word in a sentence. More generally, attention encodes vectors called token embeddings across a fixed-width sequence that can range from tens to millions of tokens in size.
Unlike "hard" weights, which are c
=====================

Implementing the memory saver

In [34]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")



In [35]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
graph_memory = builder.compile(checkpointer= memory)

In [36]:
config = {
    "configurable":{"thread_id":"1"}
    
}

In [37]:
from langchain_core.messages import HumanMessage

messages = graph_memory.invoke({"messages":HumanMessage(content = "What is 10 plus 10?")}, config= config)

for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is 10 plus 10?
================================== Ai Message ==================================
Tool Calls:
  add (call_zCnmr6kNNJ2XYEpS048ky7Hg)
 Call ID: call_zCnmr6kNNJ2XYEpS048ky7Hg
  Args:
    a: 10
    b: 10
================================= Tool Message =================================
Name: add

20
================================== Ai Message ==================================

10 plus 10 is 20.


In [38]:
from langchain_core.messages import HumanMessage

messages = graph_memory.invoke({"messages":HumanMessage(content = "Now add 5 in the previous answer?")}, config= config)

for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is 10 plus 10?
================================== Ai Message ==================================
Tool Calls:
  add (call_zCnmr6kNNJ2XYEpS048ky7Hg)
 Call ID: call_zCnmr6kNNJ2XYEpS048ky7Hg
  Args:
    a: 10
    b: 10
================================= Tool Message =================================
Name: add

20
================================== Ai Message ==================================

10 plus 10 is 20.
================================ Human Message =================================

Now add 5 in the previous answer?
================================== Ai Message ==================================
Tool Calls:
  add (call_zGcmGCixNg3P6N7iV6gep1Fj)
 Call ID: call_zGcmGCixNg3P6N7iV6gep1Fj
  Args:
    a: 20
    b: 5
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================

Adding 5 to t